# 「PyTorch Distributedの概要」

【原題】PyTorch Distributed Overview

【原著】[Shen Li](https://mrshenli.github.io/)

【元URL】https://pytorch.org/tutorials/beginner/dist_overview.html#

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年11月14日

【チュトーリアル概要】

本チュートリアルでは、`torch.distributed` パッケージの概要について説明します。

今回は実装コードはなく、解説文章のみです。

`torch.distributed` に関しては、多くのドキュメント、サンプル例、チュートリアルが各所に存在しています。

ですが、各ドキュメントやチュートリアルが、何を課題対象にしたものなのか、また、どの順番でそれらのコンテンツを読み進めれば良いのか、という点が曖昧になっています。

本チュートリアルの目標は、それらのドキュメントを数個に整理し、各トピックについて簡潔に説明することです。

PyTorchを用いた並列分散が初めての方は、そのガイドブックとして、本ドキュメントをご利用いただければと思います。

---


## 導入

PyTorch v1.6.0以降において、`torch.distributed` の機能は、以下に示す3つの主要なコンポーネントに整理されます。

- [データ並列訓練](https://pytorch.org/docs/master/generated/torch.nn.parallel.DistributedDataParallel.html)(DDP：DistributedDataParallel) は、単一プログラム複数データの訓練方法の枠組みとして広く知られています。
  DDPでは、モデルが各プロセスに複製され、複製された各モデルに対して、異なる入力データサンプルが与えられます。
  複製したモデルを同期するために勾配の情報を共有し、その情報を重ね合わせて勾配演算を行うことにより、訓練速度の向上を図ります。

- [RPC型分散訓練](https://pytorch.org/docs/master/rpc.html)(RPC)は、分散パイプライン並列化、パラメーターサーバーパラダイム、及びその他の訓練パラダイムと組み合わせたDDPなど、データ並列型訓練には収まりきらない、汎用的な訓練構造をサポートするために開発されました。
  RPC型分散訓練は、リモートに存在するオブジェクトの存続を管理し、マシンの境界を超えて自動微分エンジンを拡張する際に役立ちます。

- [集合通信](https://pytorch.org/docs/stable/distributed.html)(c10d)ライブラリは、グループ内のプロセス間でテンソルを送信できるようにサポートしています。
  c10dは、集合通信API（例：[all_reduce](https://pytorch.org/docs/stable/distributed.html#torch.distributed.all_reduce)、[all_gather](https://pytorch.org/docs/stable/distributed.html#torch.distributed.all_gather)）とP2P通信API（例：[send](https://pytorch.org/docs/stable/distributed.html#torch.distributed.send)、[isend](https://pytorch.org/docs/stable/distributed.html#torch.distributed.isend)）の両方を提供しています。
  v1.6.0の時点では、DDPとRPC([ProcessGroupバックエンド](https://pytorch.org/docs/master/rpc.html#process-group-backend))はc10d上に構築されており、前者は集合通信、後者はP2P通信を利用しています。
  上述のDDPとRPCの機能は、多くの分散型訓練のシナリオに対応することができるため、通常であれば、開発者がこれらの通信APIそのものを直接使用する必要はありません。
  しかし、これらのAPIが役立つユースケースも存在します。
  例えば、アプリケーションがDDPを用いて勾配情報を通信する代わりに、バックワードパス後にすべてのモデルのパラメーターの平均値を計算する場合には、分散パラメーター平均化を使用します。 
  これらのAPIを使用する場合、通信を演算処理から切り離し、通信の内容をより細かく制御できるようになりますが、一方でDDPが提供するパフォーマンスの最適化については断念することになります
  後ほどのチュートリアル「PyTorchで記述する分散アプリケーション」（WRITING DISTRIBUTED APPLICATIONS WITH PYTORCH）ではc10d通信APIの使用例を紹介しています。

<br>

なお、本チュートリアルの以降は、DDPかRPCのいずれかについて書かれています。

これら2つのコンポーネントに関する情報を詳細に説明します。

## データ並列訓練

PyTorchは、データ並列訓練を行うための複数の選択肢を用意しています。

単純なものから複雑なものへと、徐々に成長していくアプリケーションや、プロトタイプから本番化に至るアプリケーションには、以下のように、共通する開発方針があるでしょう。

1. データとモデルが一つのGPUで収まり、訓練速度に懸念が無いのであれば、シングルマシ・単一GPUで訓練を実施します。
2. サーバー上に複数のGPUが存在し、最小限のコード変更で訓練速度を速くしたい場合には、シングルマシン・マルチGPUで [DataParallel](https://pytorch.org/docs/master/generated/torch.nn.DataParallel.html) を使用します。
3. 追加でコードを書いて準備することを厭わず、さらに訓練速度を速くしたい場合には、シングルマシン・マルチGPUで [DistributedDataParallel](https://pytorch.org/docs/master/generated/torch.nn.parallel.DistributedDataParallel.html) を使用します。
4. アプリケーションがマシンの境界を越えてスケールする必要がある場合には、マルチマシンで [DistributedDataParallel](https://pytorch.org/docs/master/generated/torch.nn.parallel.DistributedDataParallel.html) と [起動スクリプト](https://github.com/pytorch/examples/blob/master/distributed/ddp/README.md) を使用します。
5. エラー(例：OOM)が予想される場合や、リソースが訓練中に動的に結合したり離脱したりする場合には、[torchelastic](https://pytorch.org/elastic) を利用して、分散訓練を開始します。

**ポイント**

データ並列訓練は[自動混合精度(AMP)](https://pytorch.org/docs/master/notes/amp_examples.html#working-with-multiple-gpus)と共に動作可能です。

---

### `torch.nn.DataParallel`

[DataParallel](https://pytorch.org/docs/master/generated/torch.nn.DataParallel.html) パッケージを利用すれば、最小限のコーディングでシングルマシン・マルチGPUによる並列化が実現できます。

たった1行、アプリケーションのコードを変更するだけです。

[オプション：データ並列化](https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html) のチュートリアルで例を確認できます。



なお、`DataParallel` はとても簡単に使用できますが、ほとんどのケースにおいてベストなパフォーマンスは提供できない点は注意が必要です。

これは、`DataParallel` の実装上、すべてのフォワードパスでモデルが複製されるため、必然的にそのシングルプロセス・マルチスレッド並列化がGILの競合に直面するためです。
より良いパフォーマンスを得るには、[DistributedDataParallel](https://pytorch.org/docs/master/generated/torch.nn.parallel.DistributedDataParallel.html) の使用を検討してください。



（日本語訳注：GILとはGlobal Interpreter Lockの略称です。複数スレッドでオブジェクトがインスタンス化できないなど、Pythonのマルチスレッド処理の速度弊害になる機能です。もちろん、GILのおかげで、マルチスレッドが簡便かつ適切に実行できます）

以下の記事もとても参考になります。

[Pythonで並列処理をするなら知っておくべきGILをできる限り詳しく調べてみた](https://qiita.com/ttiger55/items/5e1d5a3405d2b3ef8f40)

---

### `torch.nn.parallel.DistributedDataParallel`

DistributedDataParallelは、[DataParallel](https://pytorch.org/docs/master/generated/torch.nn.DataParallel.html) や [DistributedDataParallel](https://pytorch.org/docs/master/generated/torch.nn.parallel.DistributedDataParallel.html) と比べると、もう1ステップ準備にひと手間が必要になります。


例えば、[init_process_group](https://pytorch.org/docs/stable/distributed.html#torch.distributed.init_process_group) を呼び出すステップです。


ですが、DDPはマルチプロセス並列化を使用するため、複製されたモデル間でGILの競合は発生しません。



さらに、モデルはフォワードパス毎ではなく、DDPの構築時にブロードキャストされるため、訓練速度の向上も図っています。

その他DDPには、パフォーマンスの最適化を図る技術がいくつか搭載されています。

詳細な説明に関しては、こちらの [DDPの論文](https://arxiv.org/abs/2006.15704)(VLDB’20) を参照してください。



DDPに関する資料は以下の通りです。:

1. [DDPについて](https://pytorch.org/docs/stable/notes/ddp.html) では、スターターの例と、その設計と実装に関する簡潔な説明が記載されています。DDPの使用が初めてである場合は、このドキュメントから始めてください。
2. [分散データ並列入門](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)（日本語チュートリアルの6_3） では、不均衡なワークロード、チェックポイントの作成、マルチデバイスモデル等の、DDPの訓練を行う際に共通して直面するような問題について説明しています。なお、DDPは、[シングルマシンでのモデル並列のベストプラクティス](https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html)（日本語チュートリアルの6_2）で説明されているシングルマシン・マルチデバイス・モデル並列化と簡単に組み合わせることができます。
3. [分散データ並列アプリケーションの起動と設定](https://github.com/pytorch/examples/blob/master/distributed/ddp/README.md) では、DDPの起動スクリプトの使用方法を説明しています。
4. Amazon AWSで構築するPyTorch Distributed Trainer では、AWS上でDDPを使用する方法について説明しています。

---

### TorchElastic

アプリケーションの複雑化と規模の拡大に伴い、障害復旧は必須要件となっています。

しかし、DDPを使用しているとOOM（Out of memory）のようなエラーに直面することが避けられないことがあります。

このような場合、DDP自体はエラーから復旧することが出来ず、もちろん基本的な `try-except` ブロックを使った場合にも復旧することは出来ません。

これは、DDPではすべてのプロセスが密接に同期して動作する必要があり、異なるプロセスで起動されたすべての `AllReduce` 通信が一致する必要があるためです。

すなわち、グループ内のプロセスの一つでOOM例外が発生した場合、非同期化( `AllReduce` 操作の不一致)に至る可能性が高く、これがクラッシュやハングアップの原因になります。

訓練中に障害が発生することが予想される場合や、リソースが動的に縮小・拡大したりする可能性がある場合については、[torchelastic](https://pytorch.org/elastic)を使用して、分散データ並列訓連を起動してください。

---

## 汎用的な分散訓練手法

多くの訓練パラダイムは、場合によっては、単なるデータの並列化に収まりません。

例えば、パラメーターサーバーパラダイム、分散パイプライン並列化、複数のオブザーバーとエージェントを用いた強化学習アプリケーションなどです。

[torch.distributed.rpc](https://pytorch.org/docs/master/rpc.html) は、汎用的な分散訓練シナリオをサポートすることを目的としています。

[torch.distributed.rpc](https://pytorch.org/docs/master/rpc.html) パッケージは4つの内容を主軸としています。

- [RPC](https://pytorch.org/docs/master/rpc.html#rpc) は、リモートワーカー上に存在する関数の実行をサポートします。
- [RRef](https://pytorch.org/docs/master/rpc.html#rref) は、リモートオブジェクトの存続を管理する際に役立ちます。参照カウントプロトコルについては、[RRefについて](https://pytorch.org/docs/master/rpc/rref.html#remote-reference-protocol) に記載されています。
- [分散自動微分](https://pytorch.org/docs/master/rpc.html#distributed-autograd-framework) は、マシン間の垣根を超え、自動微分エンジンを拡張します。詳細は、[分散自動微分の設計](https://pytorch.org/docs/master/rpc/distributed_autograd.html#distributed-autograd-design) を参照してください。
- [分散最適化](https://pytorch.org/docs/master/rpc.html#module-torch.distributed.optim) は、パラメーターを更新する際に、分散自動微分エンジンによって計算された勾配を用いて、参加しているすべてのワーカーと自動で連携を取ります 。

RPCのチュートリアルは以下のとおりです。

1. [分散RPCフレームワーク入門](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html) （日本語チュートリアルの6_2）は、RPCとRRefの説明する上で、初めに単純な強化学習(RL)を扱います。その後、基本的な分散モデル並列化をRNNの例に適用し、分散自動微分と分散最適化の使い方を示します。
2. [分散RPCフレームワークを用いたパラメーターサーバーの実装](https://pytorch.org/tutorials/intermediate/rpc_param_server_tutorial.html)（日本語チュートリアルの6_6）では、[HogWild!の訓練方法](https://people.eecs.berkeley.edu/~brecht/papers/hogwildTR.pdf) に根ざし、非同期パラメーターサーバー(PS)訓練アプリケーションを扱います。
3. [RPCを用いた分散パイプライン並列化](https://pytorch.org/tutorials/intermediate/dist_pipeline_parallel_tutorial.html) （日本語チュートリアルの6_7）では、（[シングルマシン・モデル並列のベストプラクティス](https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html)で紹介されている）シングルマシン・パイプライン並列化の例を分散環境に拡張し、RPCを用いて分散パイプライン並列化を実装する方法を説明します。
4. [非同期実行を用いたRPCバッチ処理の実装](https://pytorch.org/tutorials/intermediate/rpc_async_execution.html) （日本語チュートリアルの6_8）では、訓練速度と推論速度に寄与する [@rpc.functions.async_execution](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.functions.async_execution) デコレータを使ってRPCバッチ処理を実装する方法を示しています。このチュートリアルでは、上掲のチュートリアル1、2で採用されているRLとPSの例と同様のものを使用しています。
5. [分散データ並列と分散RPCフレームワークの連携](https://pytorch.org/tutorials/advanced/rpc_ddp_tutorial.html) （日本語チュートリアルの6_9）では、分散データ並列化と分散モデル並列化を組み合わせることで、モデルを訓練する上でDDPとRPCをどのように連携できるかを解説します。

## PyTorch Distributedの開発者

PyTorch Distributedへの貢献を希望する方は、[開発者ガイド](https://github.com/pytorch/pytorch/blob/master/torch/distributed/CONTRIBUTING.md)を参照してください。